In [1]:
print("BEGIN LSI-train")
if __name__ != "__main__":
    exit()

In [1]:
# REF: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python

In [1]:
import time                       # performance/completion feedback

In [1]:
#%run ./common_preprocessor.ipynb

# folder and filename specified by client
def lsi_train_filename():
    _LSI_FILE  = 'lsi__arg'
    _LSI_FILE +='-dblim=' + str(USERARG_RECORD_LIMITER) 
    _LSI_FILE += '-num=' + str(USERARG_NUMTOPICS)
    _LSI_FILE += '-min=' + str(USERARG_NGRAM_MINCOUNT)
    _LSI_FILE += '-thr=' + str(USERARG_NGRAM_THRESHOLD)
    return _LSI_FILE


# COPIED FROM SANITIZE
def get_list_of_wordlists():
    print("\tCompiling list of word-lists from database[FIELD_BzNDX_ABSTRACT_WORDS]...")
    iTime = time.time()
    return [ record[FIELD_BzNDX_ABSTRACT_WORDS] for record in database[0:] ]


In [1]:
print("\tImporting packages...")
iTime = time.time()

import time
import gensim
#import gensim.corpora as corpora
from gensim.models import CoherenceModel, LsiModel #, HdpModel, LdaModel
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

print("\tCompleted in", round(time.time() - iTime,3),"seconds")

In [1]:
print("\tCreating Latent Semantic Indeixing model (LSI)...")
iTime = time.time()

# "LSI stands for Latent Semantic Indeixing - it is a 
# popular information retreival method which works by
# decomposing the original matrix of words to maintain
# key topics." Gensim's implementation uses an SVD.

#corpus_i2w = created by featurize.ipthnb
#ngrams   = [record[FIELD_BzNDX_ABSTRACT_WORDS] for record in database[0:]]
ngrams = get_list_of_wordlists()

model_lsi = LsiModel(corpus=corpus_i2w, num_topics=USERARG_NUMTOPICS, id2word=id2words)

# Compute Coherence
lsi_topics = model_lsi.show_topics(formatted=False)
lsi_topic_keywords = [[word for word, prob in topic] for topicid, topic in lsi_topics]
lsi_coherence = CoherenceModel(topics=lsi_topic_keywords, texts=ngrams, dictionary=id2words, window_size=10).get_coherence()
print("\tCoherence score: ", lsi_coherence)

ngrams=None

# display topics
for i in range(0,USERARG_NUMTOPICS):
    print("\tTopic",i,":",model_lsi.print_topic(i), "\n")

print("\tCompleted in", round(time.time() - iTime,3),"seconds")

In [1]:
print("\tLSI topic analysis...")
iTime = time.time()

#flatten topic_keywords
unique_topic_keywords = []
for topic in lsi_topic_keywords:
    for keyword in topic:
        if keyword not in unique_topic_keywords:
            unique_topic_keywords.append(keyword)
#print(unique_topic_keywords)
    
# visualize topics
print("\tTopicKeyword,RawTopicID", str(list(range(USERARG_NUMTOPICS))))
topic_matrix = ""
found = ""
for word in unique_topic_keywords:
    topic_matrix = "\t\t" + topic_matrix + word 
    for topic in lsi_topics:
        found = ""
        for topic_tuple in topic[1]: 
            if topic_tuple[0] == word:
                found = topic_tuple
                break
        
        if found:
            topic_matrix = topic_matrix + "," + str(found[1])
        else:
            topic_matrix = topic_matrix + ","

    topic_matrix = topic_matrix + "\n"

print("\tLSI Results (keyword vs topic#)",topic_matrix)

print("\tCompleted in", round(time.time() - iTime,3),"seconds")

In [1]:
print("\tSaving results...")
iTime = time.time()

# save reports
_TMP_FILE = 'reports/' + lsi_train_filename() + '_topics.csv'
with open(_TMP_FILE, 'w') as filehandle:
    filehandle.writelines("TopicKeyword," + str(list(range(USERARG_NUMTOPICS))) + "\n")
    filehandle.writelines(topic_matrix)
    print('\tINFO: LSI topic visualization saved to ' + _TMP_FILE)

_TMP_FILE = 'reports/' + lsi_train_filename() + '_topics.lst'
with open(_TMP_FILE, 'w') as filehandle:
    filehandle.writelines("%s\n" % [topic] for topic in lsi_topics)
    print('\tINFO: LSI topic listing saved to ' + _TMP_FILE)

# save model
model_lsi.save('models/' + lsi_train_filename() + '.model')
    
print("\tCompleted in", round(time.time() - iTime,3),"seconds")

In [1]:
print("END LSI-train")